# **Practical assignment for Topic 5**



In your assigmnet, you will use the following **data** (*do not modify the code*):

In [ ]:
### DO NOT MODIFY THIS CODE. JUST RUN IT.

from sklearn.datasets import make_blobs

# creating the dataset
X, y = make_blobs(n_samples=70, centers=[(0, 0), (3, 4), (2, -5), (6, 3), (8, -1), (6, -6)], n_features=2, cluster_std=1.5, random_state=42)
y[y == 2] = 1
y[y == 3] = 0
y[y == 4] = 1
y[y == 5] = 0

# standardizing the dataset
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

Your assignment consists of **two tasks**:

**Task 1**
* **Step 1.** Plot the dataset in a 2D plot.
* **Step 2.** Use 10-fold cross-validation to compute macro-averaged F1-score and Accuracy measure evaluating 32 models that you get by creating polynomials from degree 1 to 4 and regularizing each of them by the following C values: `[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]` (don't worry if you get a couple of warning messages about convergence). Print all 32 F1-scores and Accuracies on screen. Store all 32 F1-scores in a 2D array.
* **Step 3.** Use the array from Step 2 to plot a single diagram showing dependence of F1 on C for all four polynomial degrees. In the diagram plot four curves (one for each polynomial degree). The horizontal axis (C) should be log-scaled.
* **Step 4.** Plot four diagrams, in each showing the data points as well as the decision thresholds for one of the four polynomial degrees when its best C value is used. If you know how to plot a heatmap, you may also plot a heatmap with Degree vs. C axes. A heatmap sometimes can be more informative.

**Task 2**
* Write conclusions about the plotted diagrams, especially the F1 vs. C diagram, and printed F1 and Accuracy values. Which degree together with which C value is the best? Remember that analyzing your results is very important.
<br>

*While doing the tasks you are allowed to use only those function libraries and only those functions that we already have used in this and previous notebooks of the course. The exception is plotting a heatmap, if you decide to do that.*

**After the tasks are done, submit this file. Do not clear it's output - all print-outs and diagrams should be left in the file.**